# Interactively run workflow calcium imaging

+ The following script outlines the steps to ingest acquired metadata (ScanImage and Scanbox) and processed data (Suite2p and CaImAn) into the DataJoint `workflow-calcium-imaging`.

+ To ingest with a completely automated workflow, see [03-automate](03-automate-optional.ipynb).

+ To load the local configuration (`dj_local_conf.json`), we will change the directory to the package root directory.

In [ ]:
import os
os.chdir('..')
import numpy as np

## `Pipeline.py`

+ This script imports relevant packages and `activates` the DataJoint `elements`.

In [ ]:
from workflow_calcium_imaging.pipeline import *

## Schema diagrams

+ The following outputs are the diagrams of the schemas comprising this workflow.

+ Please refer back to these diagrams to visualize the relationships of different tables.

In [ ]:
dj.Diagram(lab)

In [ ]:
dj.Diagram(subject)

In [ ]:
dj.Diagram(session)

In [ ]:
dj.Diagram(scan)

In [ ]:
dj.Diagram(imaging)

## Insert an entry into `subject.Subject`

In [ ]:
subject.Subject.insert1(dict(subject='subject3', 
                             sex='F', 
                             subject_birth_date='2020-01-01 00:00:01', 
                             subject_description='Scanbox acquisition. Suite2p processing.'))

## Insert an entry into `lab.Equipment`

In [ ]:
Equipment.insert1(dict(scanner='Scanbox'))

## Insert an entry into `session.Session`

In [ ]:
session.Session.insert1(dict(subject='subject3', 
                             session_datetime='2021-04-30 12:22:15.032'))

## Insert an entry into `session.SessionDirectory`

+ The `session_dir` is the `imaging_root_data_dir` concatenated with the directory for the given session.

In [ ]:
session.SessionDirectory.insert1(dict(subject='subject3', 
                                      session_datetime='2021-04-30 12:22:15.032', 
                                      session_dir='<imaging_root_data_dir>/subject3/session0'))

## Insert an entry into `scan.Scan`

In [ ]:
scan.Scan.insert1(dict(subject='subject3', 
                       session_datetime='2021-04-30 12:22:15.032', 
                       scan_id=0, 
                       scanner='Scanbox', 
                       acq_software='Scanbox',
                       scan_notes=''))

## Populate `scan.ScanInfo`

+ This imported table stores information about the acquired image (e.g. image dimensions, file paths, etc.).
+ `populate` automatically calls `make` for every key for which the auto-populated table is missing data.
+ `populate_settings` passes arguments to the `populate` method.
+ `display_progress=True` reports the progress bar

In [ ]:
populate_settings = {'display_progress': True}

In [ ]:
scan.ScanInfo.populate(**populate_settings)

## (Optional) Insert a new entry into `imaging.ProcessingParamSet` for Suite2p or CaImAn

+ Define and insert the parameters that will be used for the Suite2p or CaImAn processing.

+ This step is not needed if you are using an existing ProcessingParamSet.

### Define Suite2p parameters

In [ ]:
params_suite2p = {'look_one_level_down': 0.0,
                  'fast_disk': [],
                  'delete_bin': False,
                  'mesoscan': False,
                  'h5py': [],
                  'h5py_key': 'data',
                  'save_path0': [],
                  'subfolders': [],
                  'nplanes': 1,
                  'nchannels': 1,
                  'functional_chan': 1,
                  'tau': 1.0,
                  'fs': 10.0,
                  'force_sktiff': False,
                  'preclassify': 0.0,
                  'save_mat': False,
                  'combined': True,
                  'aspect': 1.0,
                  'do_bidiphase': False,
                  'bidiphase': 0.0,
                  'do_registration': True,
                  'keep_movie_raw': False,
                  'nimg_init': 300,
                  'batch_size': 500,
                  'maxregshift': 0.1,
                  'align_by_chan': 1,
                  'reg_tif': False,
                  'reg_tif_chan2': False,
                  'subpixel': 10,
                  'smooth_sigma': 1.15,
                  'th_badframes': 1.0,
                  'pad_fft': False,
                  'nonrigid': True,
                  'block_size': [128, 128],
                  'snr_thresh': 1.2,
                  'maxregshiftNR': 5.0,
                  '1Preg': False,
                  'spatial_hp': 50.0,
                  'pre_smooth': 2.0,
                  'spatial_taper': 50.0,
                  'roidetect': True,
                  'sparse_mode': False,
                  'diameter': 12,
                  'spatial_scale': 0,
                  'connected': True,
                  'nbinned': 5000,
                  'max_iterations': 20,
                  'threshold_scaling': 1.0,
                  'max_overlap': 0.75,
                  'high_pass': 100.0,
                  'inner_neuropil_radius': 2,
                  'min_neuropil_pixels': 350,
                  'allow_overlap': False,
                  'chan2_thres': 0.65,
                  'baseline': 'maximin',
                  'win_baseline': 60.0,
                  'sig_baseline': 10.0,
                  'prctile_baseline': 8.0,
                  'neucoeff': 0.7,
                  'xrange': np.array([0, 0]),
                  'yrange': np.array([0, 0])}

### Insert Suite2p parameters

In [ ]:
imaging.ProcessingParamSet.insert_new_params(
    processing_method='suite2p', 
    paramset_idx=0, 
    params=params_suite2p,
    paramset_desc='Calcium imaging analysis with Suite2p using default Suite2p parameters')

## Insert new ProcessingTask to trigger ingestion of motion-correction/segmentation results

+ Motion correction and segmentation are performed for each scan,
once the processing job has been completed, an entry in the `ProcessingTask` needs to be added to trigger the ingestion of the processed results.

+ Two pieces of information need to be specified:

    + The `paramset_idx` is the parameter set stored in `imaging.ProcessingParamSet` that is used for the imaging processing job.
    
    + The `processing_output_dir` stores the directory of the processing results (relative to the imaging root data directory).

In [ ]:
imaging.ProcessingTask.insert1(dict(subject='subject3', 
                                    session_datetime='2021-04-30 12:22:15.032', 
                                    scan_id=0,
                                    paramset_idx=0,
                                    processing_output_dir='subject3/session0',
                                    task_mode='load'))

## Populate `imaging.Processing`

+ For the `task_mode=load` specified above in `imaging.ProcessingTask`, this step ensures that the output directory contains the valid processed outputs.

+ In the future, this step will provide for the option to `trigger` the analysis within this workflow (if the `task_mode=trigger`).

In [ ]:
imaging.Processing.populate(**populate_settings)

## Insert new Curation following the ProcessingTask

+ The next step in the pipeline is the curation of segmentation results. If a manual curation was implemented, an entry needs to be manually inserted into the table Curation, which specifies the directory to the curated results in curation_output_dir. If we would like to process the processed outcome directly, an entry is also needed in Curation. A method create1_from_processing_task was provided to help this insertion. It copies the processing_output_dir in ProcessingTask to the field curation_output_dir in the table Curation with a new curation_id.

+ In this example, we create/insert one `imaging.Curation` for each `imaging.ProcessingTask`, specifying the same output directory.

+ To this end, we could also make use of a convenient function `imaging.Curation().create1_from_processing_task()`

In [ ]:
imaging.Curation.insert1(dict(subject='subject3', 
                              session_datetime='2021-04-30 12:22:15.032', 
                              scan_id=0,
                              paramset_idx=0,
                              curation_id=0,
                              curation_time='2021-04-30 12:22:15.032', 
                              curation_output_dir='subject3/session0',
                              manual_curation=False,
                              curation_note=''))

## Populate `imaging.MotionCorrection`

+ This table contains the rigid or non-rigid motion correction data including the shifts and summary images.


In [ ]:
imaging.MotionCorrection.populate(**populate_settings)

## Populate `imaging.Segmentation`

+ This table contains the mask coordinates, weights, and centers.

In [ ]:
imaging.Segmentation.populate(**populate_settings)

## Insert an entry into `imaging.Curation`
    
+ As an option, you can create a new entry for a curation of the motion correction or segmentation data.

+ If adding a new entry to `imaging.Curation`, please populate the `imaging.MotionCorrection` and `imaging.Segmentation` tables.

## Populate `imaging.MaskClassification`

+ This table is currently not implemented.

In [ ]:
imaging.MaskClassification.populate(**populate_settings)

## Populate `imaging.Fluorescence`

+ This table contains the fluorescence traces prior filtering and spike extraction

In [ ]:
imaging.Fluorescence.populate(**populate_settings)

## Populate `imaging.Activity`
+ This table contains the inferred neural activity from the fluorescence traces.

In [ ]:
imaging.Activity.populate(**populate_settings)

## Next steps

+ Proceed to the [05-explore](05-explore.ipynb) to learn how to  query, fetch, and visualize the imaging data.